## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
fine_tune_openai = os.getenv('FINE_TUNED_MODEL')
google_api_key = os.getenv('GOOGLE_API_KEY')

groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Groq API Key exists and begins gsk_


In [4]:
# Agent 1 is the coordinator and interact with customer, Agent 2 is a tool can compute the repayment probability score and recommendations, 
# Agent 3 give more detail analysis of result from agent 2 
# Agent 4 is send email to customer with loan offer
# 
# 
#  Agent1: Manage conversation with customer and look for key words to determine if the customer is interested in a loan. 
# If customer is interested send input data structure customer age, marital status, location(eg Lagos, abuja, oyo,etc), amount, and tenure. etc.
#  Validate if use has enter all these values if yes call agent 2:
#  Agent  3 handoff to agent 1, it ask user to confirm loan offer and if yes handoff to agent 4

#Agent 2: Uses my finetune OpenAi model to compute the repayment probability score and recommendations

#Agent 3: Elaborate on the recommendations and ask for the final go ahead for the loan handoff to agent 1

#Agent 4: Send the final loan offer to the customer buy using tools to send email

In [5]:
# Define instructions for the coordinator agent
instructions1 = """
You are a Nigeria loan application coordinator officer. Your job is to collect and validate information from users who want to apply for a loan.

Follow these steps:
1. Collect all required information for the loan application:
   - Age (must be between 19-99)
   - Gender (must be male, female, or other)
   - Marital status (must be single, married, divorced, or widowed)
   - Location (state where the applicant lives in Nigeria)
   - Loan amount (must be greater than 0 and not exceed 1,000,000)
   - Loan tenure in days (must be greater than 6 and not exceed 180 days)

2. Keep track of what information has been provided and what's still missing.

3. When a user provides a value, validate it according to the requirements and ask for corrections if needed.

4. After collecting all required information, summarize the complete application and ask the user to confirm before proceeding.

5. If the user wants to modify any information, allow them to do so before final confirmation.

Always be friendly, professional, and helpful throughout the process.
"""

# Define instructions for the processing agent (agent two)
instructions2 = """
You are a loan application processor. Your job is to review completed loan applications and make decisions.

You will receive a complete loan application with all required fields. Review the application carefully and provide an initial assessment.

Consider factors such as:
- Appropriate loan amount based on standard lending practices
- Suitable tenure period
- Any potential risk factors

Provide a professional response indicating whether the application looks promising or might need adjustments.
"""





### It's easy to use any models with OpenAI compatible endpoints

In [6]:
import sys
import os

sys.path.append(os.path.abspath("..")) 

from agents import Agent, Runner, trace, function_tool
from lab.utils.NLExtractor import ApplicationExtractor
from lab.utils.ValidationModels import LoanApplicationValidator
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import Optional, Dict, Any
import gradio as gr
import json


# Create the Coordinator agent
coordinator_agent1 = Agent(
    name="Loan Application Coordinator",
    model="gpt-4o-mini",  
    instructions=instructions1,
    tools=[],
)



# Create the Processing agent
# repayment_predictor_agent = Agent(
#     name="Repayment Probability Agent",
#     model=fine_tune_openai,
#     instructions=instructions2,
#     tools=[]
# )

# Create the Processing agent
processor_agent = Agent(
    name="Loan Application Processor",
    model="gpt-4o-mini",
    instructions=instructions2,
    tools=[]
)

# Function to store application data in session state
def initialize_session_state():
    return {
        "application_data": {},
        "fields_collected": set(),
        "required_fields": {"age", "gender", "marital_status", "location", "amount", "tenure"},
        "confirmation_stage": False,
        "extractor": ApplicationExtractor()
    }

# Format the collected application data for display
def format_application_data(app_data, fields_collected, required_fields):
    summary = "### Application Progress\n\n"
    
    # Add collected fields
    if fields_collected:
        summary += "**Collected Information:**\n"
        for field in fields_collected:
            if field in app_data:
                summary += f"- {field.replace('_', ' ').title()}: {app_data[field]}\n"
        summary += "\n"
    
    # Add missing fields
    missing = required_fields - fields_collected
    if missing:
        summary += "**Information Still Needed:**\n"
        for field in missing:
            summary += f"- {field.replace('_', ' ').title()}\n"
    
    return summary


# Update the chat function to use the improved extractor
async def chat(message, history, session_state):
    if session_state is None:
        session_state = initialize_session_state()
        # Initialize the extractor if it doesn't exist
        session_state["extractor"] = ApplicationExtractor()
    
    history = history or []
    
    # If we're in confirmation stage and user confirms, process with agent two
    if session_state["confirmation_stage"] and any(confirm in message.lower() for confirm in ["confirm", "yes", "correct", "submit", "proceed"]):
        try:
             validated = LoanApplicationValidator(**session_state["application_data"])
        except ValidationError as e:
            return "", history + [(message, f"Validation Error:\n{str(e)}")], session_state
            
        # Call the second agent with the complete application data
        with trace("Loan processor"):
            app_data_str = json.dumps(session_state["application_data"])
            run_result = await Runner.run(
                processor_agent, 
                f"Please review this loan application: {app_data_str}"
            )
            processor_response = run_result.final_output
        
        # Reset the session state for a new application
        session_state = initialize_session_state()
        session_state["extractor"] = ApplicationExtractor()
        
        # Add the processor's response to the history
        history.append((message, processor_response))
        return "", history, session_state
    
    # If in confirmation stage but user wants to modify, exit confirmation stage
    elif session_state["confirmation_stage"] and any(modify in message.lower() for modify in ["modify", "change", "edit", "no", "incorrect"]):
        session_state["confirmation_stage"] = False
        response = "What information would you like to modify?"
        history.append((message, response))
        return "", history, session_state
    
    # Extract information using the NLP extractor
    missing_fields = session_state["required_fields"] - session_state["fields_collected"]
    extracted_data = session_state["extractor"].extract_all_fields(
        message, 
        missing_fields,
        session_state["application_data"]
    )
    
    # Update session state with extracted information
    for field, (value, confidence) in extracted_data.items():
        session_state["application_data"][field] = value
        session_state["fields_collected"].add(field)
    
    # Normal flow - collect information or process with agent one
    with trace("Loan manager"):
        # Append the current application state to the message
        app_summary = format_application_data(
            session_state["application_data"], 
            session_state["fields_collected"], 
            session_state["required_fields"]
        )
        
        # Add information about newly extracted fields to help the agent frame its response
        if extracted_data:
            newly_extracted = "\n\n[SYSTEM INFO: Newly extracted fields]\n"
            for field, (value, confidence) in extracted_data.items():
                newly_extracted += f"- {field}: {value} (confidence: {confidence:.2f})\n"
            enhanced_message = f"{message}\n\n[SYSTEM INFO: Current application state]\n{app_summary}{newly_extracted}"
        else:
            enhanced_message = f"{message}\n\n[SYSTEM INFO: Current application state]\n{app_summary}"
        
        run_result = await Runner.run(coordinator_agent1, enhanced_message)
        print('run_result', run_result)
        agent_response = run_result.final_output
        
        # Check if all required fields are collected
        if session_state["fields_collected"] == session_state["required_fields"] and not session_state["confirmation_stage"]:
            # Enter confirmation stage
            session_state["confirmation_stage"] = True
            
            # Prepare confirmation message
            confirmation = "### Complete Application Summary\n\n"
            for field, value in session_state["application_data"].items():
                confirmation += f"- **{field.replace('_', ' ').title()}**: {value}\n"
            confirmation += "\nIs this information correct? Please confirm to proceed or say 'modify' to make changes."
            
            # Append to response
            agent_response += f"\n\n{confirmation}"
    
    history.append((message, agent_response))
    return "", history, session_state



with gr.Blocks() as demo:
    session_state = gr.State(initialize_session_state())
    
    gr.Markdown("# Loan Application System")
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=500)
            
            with gr.Row():
                msg = gr.Textbox(
                    label="Your message", 
                    show_label=False,
                    placeholder="Type your message here...",
                    container=False
                )
                submit_btn = gr.Button("Submit")
    
        with gr.Column(scale=1):
            app_status = gr.Markdown("### Application Status\nStart by providing your information")
    
    with gr.Row():
        clear_btn = gr.Button("Start New Application")
    
    # Update application status display when session state changes
    def update_status(session_state):
        if session_state is None:
            return "### Application Status\nStart by providing your information"
        
        return format_application_data(
            session_state["application_data"],
            session_state["fields_collected"],
            session_state["required_fields"]
        )
    
    # Handle interactions
    submit_btn.click(chat, [msg, chatbot, session_state], [msg, chatbot, session_state])
    msg.submit(chat, [msg, chatbot, session_state], [msg, chatbot, session_state])
    
    # Update application status whenever session state changes
    session_state.change(update_status, session_state, app_status)
    
    # Clear chat and reset session
    clear_btn.click(
        lambda: (None, initialize_session_state()), 
        None, 
        [chatbot, session_state],
        queue=False
    )

if __name__ == "__main__":
    demo.launch()

/var/folders/5t/v0zfcvkd3zdckpwz0d5vfk200000gn/T/ipykernel_95112/1270579563.py:174: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [7]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [8]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

NameError: name 'deepseek_api_key' is not defined

In [7]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [8]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [9]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [14]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [15]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [16]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [20]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>